In [ ]:
from flightmech.aircraft import Aircraft
from flightmech.propeller import Propeller

prop = Propeller(d=2, zeta=0.7, P_max=90e3)
ac = Aircraft(S=197, b=65, prop=prop, n_prop=2, C_A_alpha=0.0947, C_A_0=0.253)

